In [2]:
import json
import pandas as pd
import os
import pickle

In [3]:
with open('data/score_mappings.json', 'r') as f:
    score_mappings = json.load(f)

In [4]:
with open('data/accounts.pkl', 'rb') as f:
    accounts = pickle.load(f)

In [5]:
for account in accounts:
    if len(account) < 3:
        account = (*account, "did not pull") 
        # original accounts lists did not have name field, 
        # so we only observe name field when it was pulled 
        # in a follower list (unfollowed politicians don't 
        # have a name field)

accounts_dict = {}
for account in accounts:
    accounts_dict[account[1]] = account

In [14]:
tweet_samples = ["new_tweets", "old_tweets"]
for sample in tweet_samples:
    df = pd.DataFrame(columns = ['tweet_id', 
    'account_id', 'username', 'name', 'score', 'quantile', 'ideology_label', 
    'text', 'retweet_count', 'reply_count', 'like_count', 
    'quote_count', 'lang', 'created_at'])
    path = "data/" + sample
    files = os.listdir(path)
    for file in files:

        account_id = file.removesuffix("_tweets.pkl")
        score =  score_mappings[account_id]["score"], 
        quantile = score_mappings[account_id]["quantile"]
        ideology = "conservative" if quantile > 4 else "liberal" if quantile < 4 else "neutral" 

        with open(path + "/" + file, "rb") as f:
            tweets_list = pickle.load(f)
        
        for tweet in tweets_list:
            row = pd.DataFrame({'tweet_id' : tweet["id"], 
            'account_id' : account_id, 
            'username' : accounts_dict[account_id][0], 
            'name' : accounts_dict[account_id][2],
            'score' : score, 
            'quantile' : quantile,
            'ideology_label' : ideology, 
            'text' : tweet["text"], 
            'retweet_count' : tweet["public_metrics"]["retweet_count"], 
            'reply_count' : tweet["public_metrics"]["reply_count"], 
            'like_count' : tweet["public_metrics"]["like_count"], 
            'quote_count' : tweet["public_metrics"]["quote_count"], 
            'lang' : tweet["lang"], 
            'created_at' : tweet["created_at"]})
            print(row)
            df = pd.concat([df, row]).reset_index(drop=True)
            print(df)
    
    with open("data/" + sample + "_df.pkl", 'wb') as f:
        pickle.dump(df, f)



              tweet_id account_id username                  name     score  \
0  1598065068863889408  153558139     NPWF  National Partnership -0.792856   

   quantile ideology_label                                               text  \
0         1        liberal  RT @JECDems: Today is #NativeWomensEqualPay Da...   

   retweet_count  reply_count  like_count  quote_count lang  \
0              5            0           0            0   en   

                 created_at  
0  2022-11-30T21:22:57.000Z  
              tweet_id account_id username                  name     score  \
0  1598065068863889408  153558139     NPWF  National Partnership -0.792856   

  quantile ideology_label                                               text  \
0        1        liberal  RT @JECDems: Today is #NativeWomensEqualPay Da...   

  retweet_count reply_count like_count quote_count lang  \
0             5           0          0           0   en   

                 created_at  
0  2022-11-30T21:22:57.000